In [ ]:
# Create dotaro cut up dataset from DOTA
# Prepare Dota
input_dir = 'output/dotain/'

In [ ]:
from DotaTools.ImgSplit import splitbase
# First CUT the images
split = splitbase(input_dir +'val/rest',
                  input_dir +'val/rest_out', thresh = 0.33)
split.splitdata(1)
split = splitbase(input_dir +'train/rest',
                  input_dir +'train/rest_out', thresh = 0.33)
split.splitdata(1)
print("Dn")

In [ ]:
# After images were cut:
# Remove empty images
import os
import shutil

if not os.path.exists(input_dir+'train/rest_through/images'):
    os.makedirs(input_dir+'train/rest_through/images')
if not os.path.exists(input_dir+'val/rest_through/images'):
    os.makedirs(input_dir+'val/rest_through/images')
if not os.path.exists(input_dir+'train/rest_through/labelTxt'):
    os.makedirs(input_dir+'train/rest_through/labelTxt')
if not os.path.exists(input_dir+'val/rest_through/labelTxt'):
    os.makedirs(input_dir+'val/rest_through/labelTxt')


def cut_empty(in_file,out_file):
    for file in os.listdir(in_file+'labelTxt'):
        file_id = file.split('.')[0]
        if os.stat(os.path.join(in_file,'labelTxt',file)).st_size != 0:
            file_path = os.path.abspath(os.path.join(in_file,'images',f'{file_id}.png'))
            file_newpath = os.path.abspath(os.path.join(out_file,'images',f'{file_id}.png'))
            shutil.copyfile(file_path, file_newpath)
            label_path = os.path.abspath(os.path.join(in_file,'labelTxt',f'{file_id}.txt'))
            label_newpath = os.path.abspath(os.path.join(out_file,'labelTxt',f'{file_id}.txt'))
            shutil.copyfile(label_path, label_newpath)

cut_empty(input_dir+'train/rest_out/',input_dir+'train/rest_through/')
cut_empty(input_dir+'val/rest_out/',input_dir+'val/rest_through/')
print("Dn")

In [ ]:
# Rename images
import os

def rename_images(dir_in,dir_img):
    visited = []
    img_visited = []
    letters = ['b','c','d','e','f','g','h','i','j','k','l','m','n','o','q','p','r','s','t','u','v','w','x','y','z','alfa','beta','gamma','delta','elipson','zeta','eta','theta','jota','kappa','lambda','mu','nu','xi','omicron','pi','ro','sigma','tau','ypsilon','fi','chi','psi','omega','alep','bet','gimi','dalet','he','waw','zayin','het','tet','yod','kap','lamed','mem','nun','samek','ayin','pe','sade','qop','res','sin','taw']

    def get_new_name(dir, file,name,end,visit):
        for l in letters:
            new_name = name + l
            if new_name in visit:
                continue
            else:
                os.rename(dir + file, dir + new_name + end)
                visit.append(new_name)
                return 0
        return 1
    #Labels
    end = '.txt'
    labels = os.listdir(dir_in)
    for file in os.listdir(dir_in):
        if '__' in file:
            name = file.split('__')[0]
        else:
            name = file.split(end)[0]
            visited.append(name)
            continue
        r = get_new_name(dir_in,file,name,end,visited)
        if r == 1:
            print("To few letters")
    # Images:
    imgend = '.png'
    for label in labels:
        file = label.split(end)[0] + imgend
        if '__' in file:
            name = file.split('__')[0]
        else:
            name = file.split(imgend)[0]
            visited.append(name)
            continue
        r = get_new_name(dir_img,file,name,imgend,img_visited)
        if r == 1:
            print("To few letters")

rename_images(input_dir+'train/rest_through/labelTxt/',input_dir+'train/rest_through/images/')
rename_images(input_dir+'val/rest_through/labelTxt/',input_dir+'val/rest_through/images/')

print("Done")

In [ ]:
def copy_files(f_in,f_out):
    if not os.path.exists(f_out):
        os.makedirs(f_out)
    fs = os.listdir(f_in)
    for f in fs:
        shutil.copyfile(os.path.abspath(os.path.join(f_in,f)),os.path.abspath(os.path.join(f_out,f)))

def link_files(f_in,f_out):
    if not os.path.exists(f_out):
        os.makedirs(f_out)
    fs = os.listdir(f_in)
    for f in fs:
        os.symlink(os.path.abspath(os.path.join(f_in,f)),os.path.abspath(os.path.join(f_out,f)))



In [ ]:
# Add airports
copy_files(input_dir+'train/airport/images/',input_dir+'train/rest_through/images/')
copy_files(input_dir+'train/airport/remap_labels/',input_dir+'train/rest_through/labelTxt/')
copy_files(input_dir+'val/airport/images/',input_dir+'val/rest_through/images/')
copy_files(input_dir+'val/airport/remap_labels/',input_dir+'val/rest_through/labelTxt/')
print("Done")

In [ ]:
# Compress Images:
import os
import shutil
def compress_images(dir_in,dir_out):
    if not os.path.exists(dir_out):
        os.makedirs(dir_out)
    command = 'for file in '+ dir_in +'*; do ffmpeg -i "$file" -vf "scale=640:640" "'+ dir_out +'$(basename -- "$file")"; done'
    os.system(command)

def change_end(in_dir,end_in,end_out):
    command = f'mogrify -format {end_out} *.{end_in} -path {in_dir}'



compress_images(input_dir+'train/rest_through/images/',input_dir+'train/rest_compressed/images/')
compress_images(input_dir+'val/rest_through/images/',input_dir+'val/rest_compressed/images/')


In [ ]:
# Change format to jpg - TODO mogrify -format jpg *.png
#change_end(input_dir+'train/rest_through/images/','png','jpg')
#change_end(input_dir+'val/rest_through/images/','png','jpg')

In [ ]:
#Copy Labels
copy_files(input_dir+'train/rest_through/labelTxt/',input_dir+'train/rest_compressed/labelTxt/')
copy_files(input_dir+'val/rest_through/labelTxt/',input_dir+'val/rest_compressed/labelTxt/')

In [ ]:
# Convert Dota dataset to the Coco
import os
from DotaTools.DOTA2COCO import dota2coco


dota2coco(srcpath_img=input_dir+'train/rest_through/images/', srcpath_labels=input_dir+'train/rest_through/labelTxt/',out_json=input_dir+'train/rest_through/segments_coco.json',end='.png')
dota2coco(srcpath_img=input_dir+'val/rest_through/images/', srcpath_labels=input_dir+'val/rest_through/labelTxt/',out_json=input_dir+'val/rest_through/segments_coco.json',end='.png')


In [ ]:
## Script convert Coco to Yolo:
import os
from JSON2YOLO.general_json2yolo import coco_to_yolo

if not os.path.exists(input_dir+'train/rest_through/yolo/'):
   os.makedirs(input_dir+'train/rest_through/yolo/')
if not os.path.exists(input_dir+'val/rest_through/yolo/'):
   os.makedirs(input_dir+'val/rest_through/yolo/')

coco_to_yolo(input_dir+'train/rest_through/segments_coco.json',input_dir+'train/rest_through/yolo/')
coco_to_yolo(input_dir+'val/rest_through/segments_coco.json',input_dir+'val/rest_through/yolo/')


In [ ]:
# Copy yolo to the comressed
if not os.path.exists(input_dir+'train/rest_compressed/yolo/'):
   os.makedirs(input_dir+'train/rest_compressed/yolo/')
if not os.path.exists(input_dir+'val/rest_compressed/yolo/'):
   os.makedirs(input_dir+'val/rest_through/yolo/')

copy_files(input_dir+'train/rest_through/yolo/',input_dir+'train/rest_compressed/yolo/')
copy_files(input_dir+'val/rest_through/yolo/',input_dir+'val/rest_compressed/yolo/')

In [ ]:
## Script convert Yolo to Coco:
import globox
yolo = globox.AnnotationSet.from_yolo_v5(
    folder=input_dir+'train/rest_compressed/yolo/',
    image_folder=input_dir+'train/rest_compressed/images/'
)
yolo.save_coco(path=input_dir+'train/rest_compressed/coco.json',label_to_id={'0':1,'1':2, '2':3, '3':4, '4':5},imageid_to_id = {key: value for key, value in zip(yolo.image_ids,
 [r for r in range(0,len(yolo.image_ids))])  })

yolov = globox.AnnotationSet.from_yolo_v5(
    folder=input_dir+'val/rest_compressed/yolo/',
    image_folder=input_dir+'val/rest_compressed/images/'
)
yolov.save_coco(path=input_dir+'val/rest_compressed/coco.json',label_to_id={'0':1,'1':2, '2':3, '3':4, '4':5},imageid_to_id = {key: value for key, value in zip(yolov.image_ids,
 [r for r in range(0,len(yolov.image_ids))])  })


import json
def fix_categories(file,out):
    with open(file,"r") as f:
        data = json.load(f)
    new_data = {}
    new_data['info'] =   {
    "year": "2023",
    "version": "1",
    "description": "Created using globox",
    "contributor": "atm",
    },
    new_data['licenses'] =  [
    {
        "id": 1,
        "url": "https://creativecommons.org/licenses/by/4.0/",
        "name": "CC BY 4.0"
    }
    ]
    new_data['categories'] = [
    {
        "id": 1,
        "name": "airport",
        "supercategory": "objects"
    },
    {
        "id": 2,
        "name": "helicopter",
        "supercategory": "objects"
    },
    {
        "id": 3,
        "name": "oiltank",
        "supercategory": "objects"
    },
    {
        "id": 4,
        "name": "plane",
        "supercategory": "objects"
    },
    {
        "id": 5,
        "name": "warship",
        "supercategory": "objects"
    }
    ]
    new_data['images'] = data['images']
    new_data['annotations'] = data['annotations']
    with open(out,'w') as fout:
        json.dump(new_data, fout , indent=4)

fix_categories(input_dir+'train/rest_compressed/coco.json',input_dir+'train/rest_compressed/coco.json')
fix_categories(input_dir+'val/rest_compressed/coco.json',input_dir+'val/rest_compressed/coco.json')

In [ ]:
## Script convert Coco to voc:
# create voc dir!
if not os.path.exists(input_dir+'train/rest_compressed/voc/'):
   os.makedirs(input_dir+'train/rest_compressed/voc/')
if not os.path.exists(input_dir+'val/rest_compressed/voc/'):
   os.makedirs(input_dir+'val/rest_compressed/voc/')

from coco_to_voc import coco2voc

coco2voc(input_dir+'train/rest_compressed/coco.json',input_dir+'train/rest_compressed/voc/',ext='.jpg')
coco2voc(input_dir+'val/rest_compressed/coco.json',input_dir+'val/rest_compressed/voc/',ext='.jpg')

In [ ]:
## Script to convert Coco to Yolified Coco

from yolifycoco import yolifycoco

yolifycoco(input_dir+'train/rest_compressed/coco.json',input_dir+'train/rest_compressed/cocoyolo.json',split='.jpg')
yolifycoco(input_dir+'val/rest_compressed/coco.json',input_dir+'val/rest_compressed/cocoyolo.json',split='.jpg')

In [ ]:
# Create Combi:
if not os.path.exists(input_dir+'train/combi/images/'):
   os.makedirs(input_dir+'train/combi/images/')
if not os.path.exists(input_dir+'train/combi/yolo/'):
   os.makedirs(input_dir+'train/combi/yolo/')
if not os.path.exists(input_dir+'val/combi/images/'):
   os.makedirs(input_dir+'val/combi/images/')
if not os.path.exists(input_dir+'val/combi/yolo/'):
   os.makedirs(input_dir+'val/combi/yolo/')


link_files(input_dir+'train/rest_compressed/images/',input_dir+'train/combi/images/')
link_files(input_dir+'train/backgrounds/images/',input_dir+'train/combi/images/')
link_files(input_dir+'train/rest_compressed/yolo/',input_dir+'train/combi/yolo/')
link_files(input_dir+'train/backgrounds/yolo/',input_dir+'train/combi/yolo/')


link_files(input_dir+'val/rest_compressed/images/',input_dir+'val/combi/images/')
link_files(input_dir+'val/backgrounds/images/',input_dir+'val/combi/images/')
link_files(input_dir+'val/rest_compressed/yolo/',input_dir+'val/combi/yolo/')
link_files(input_dir+'val/backgrounds/yolo/',input_dir+'val/combi/yolo/')

print("Done")

In [ ]:
## Script combi convert Yolo to Coco:
import globox
yolo = globox.AnnotationSet.from_yolo_v5(
    folder=input_dir+'train/combi/yolo/',
    image_folder=input_dir+'train/combi/images/'
)
yolo.save_coco(path=input_dir+'train/combi/combi_coco.json',label_to_id={'0':1,'1':2, '2':3, '3':4, '4':5},imageid_to_id = {key: value for key, value in zip(yolo.image_ids,
 [r for r in range(0,len(yolo.image_ids))])  })

yolov = globox.AnnotationSet.from_yolo_v5(
    folder=input_dir+'val/combi/yolo/',
    image_folder=input_dir+'val/combi/images/'
)
yolov.save_coco(path=input_dir+'val/combi/combi_coco.json',label_to_id={'0':1,'1':2, '2':3, '3':4, '4':5},imageid_to_id = {key: value for key, value in zip(yolov.image_ids,
 [r for r in range(0,len(yolov.image_ids))])  })


import json
def fix_categories(file,out):
    with open(file,"r") as f:
        data = json.load(f)
    new_data = {}
    new_data['info'] =   {
    "year": "2023",
    "version": "1",
    "description": "Created using globox",
    "contributor": "atm",
    },
    new_data['licenses'] =  [
    {
        "id": 1,
        "url": "https://creativecommons.org/licenses/by/4.0/",
        "name": "CC BY 4.0"
    }
    ]
    new_data['categories'] = [
    {
        "id": 1,
        "name": "airport",
        "supercategory": "objects"
    },
    {
        "id": 2,
        "name": "helicopter",
        "supercategory": "objects"
    },
    {
        "id": 3,
        "name": "oiltank",
        "supercategory": "objects"
    },
    {
        "id": 4,
        "name": "plane",
        "supercategory": "objects"
    },
    {
        "id": 5,
        "name": "warship",
        "supercategory": "objects"
    }
    ]
    new_data['images'] = data['images']
    new_data['annotations'] = data['annotations']
    with open(out,'w') as fout:
        json.dump(new_data, fout , indent=4)

fix_categories(input_dir+'train/combi/combi_coco.json',input_dir+'train/combi/combi_coco.json')
fix_categories(input_dir+'val/combi/combi_coco.json',input_dir+'val/combi/combi_coco.json')



In [ ]:
# Yolify combi COCO
from yolifycoco import yolifycoco

yolifycoco(input_dir+'train/combi/combi_coco.json',input_dir+'train/combi/combi_cocoyolo.json',split='.jpg')
yolifycoco(input_dir+'val/combi/combi_coco.json',input_dir+'val/combi/combi_cocoyolo.json',split='.jpg')

In [ ]:
#P0259